In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

In [2]:
apiUrl = 'https://api.pro.coinbase.com'

In [3]:
#5min frequency
barSize = '300'
timeEnd = datetime.now()
delta = timedelta(seconds = int(barSize))
print(delta)
#105120 records for 1 year data
timeStart = timeEnd - (105120*delta)
print(timeEnd)
print(timeStart)

0:05:00
2022-02-19 23:44:13.843527
2021-02-19 23:44:13.843527


In [4]:
timeStart = timeStart.isoformat()
timeEnd = timeEnd.isoformat()
timeStart


'2021-02-19T23:44:13.843527'

In [5]:
timeEnd

'2022-02-19T23:44:13.843527'

In [6]:
datf = pd.DataFrame([])

In [7]:
#Get all 5 min frequencyy time stamps

l = (pd.DataFrame(columns=['NULL'],
                  index=pd.date_range(timeStart, timeEnd,
                                      freq='25H'))
       .between_time('00:00','23:59')
       .index.strftime('%Y-%m-%dT%H:%M:%SZ')
       .tolist()
)

In [8]:
#verify

from itertools import tee, islice, chain

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

#for previous, item, nxt in previous_and_next(l):
    #print("Time is now", item, "next is", nxt, "previous is", previous)

In [9]:
#Add or update one or more symbols
#Add for loop for symbols more than one.
sym = 'ETC'
for previous, item, nxt in previous_and_next(l):
    parameters = {
        'start':item,
        'end': nxt,
        'granularity':barSize,
    }

    data = requests.get(f"{apiUrl}/products/{sym}-USD/candles",
                       params =parameters,
                       headers = {'content-type':'application/json'})
    df = pd.DataFrame(data.json(),
                     columns = ['time','low','high','open','close','volume'])
    df['date'] = pd.to_datetime(df['time'],unit='s')
    df = df[['date', 'open', 'high','close','volume']]
    df['symbol'] = sym
    datf = datf.append(df)
    
datf
datf.to_csv('ETC1.csv', index=False)